# SQL script to read from products silver tables and join them into one dim products table

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY cp.start_date, cp.product_key) AS product_key, -- Surrogate key
    cp.product_id,
    cp.product_key AS product_number,
    cp.product_name,
    cp.product_cost,
    cp.product_line,
    cp.start_date,
    cp.end_date,
    cp.category_id,
    pc.category,
    pc.subcategory,
    pc.maintenance_flag
FROM workspace.silver.crm_products cp
LEFT JOIN workspace.silver.erp_product_category pc
    ON cp.category_id = pc.category_id
"""

df = spark.sql(query)

In [0]:
df.limit(10).display()

# writing to gold table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_products")

# sanity check for gold table

In [0]:
%sql
SELECT * FROM workspace.gold.dim_products LIMIT 10